In [ ]:
!pip install sentencepiece

In [ ]:
import os
import torch
from sentencepiece import SentencePieceProcessor
import json
import random
import copy
from sklearn.model_selection import train_test_split


In [ ]:
spm=SentencePieceProcessor(model_file="pytest.model")

In [ ]:
n_dec=512 #디코더 모델의 크기
def makebatch(datafile,outfile,sid):
  qs=[]
  ans=[]
  lines = open(datafile, 'r').readlines()
  id=sid
  with open(outfile, "w") as f:
    for line in lines:
      ans=spm.encode_as_ids(line)
      l=len(ans)
      if(l>n_dec-2):
        ans=ans[:n_dec-2]
      ans.insert(0,2) #vocab 상 2=bos 3= eos 6=mask
      ans.append(3)
      ran=random.randrange(1,l+2)
      qs=copy.deepcopy(ans)
      qs[ran]=6
      datablock={"ids":id,"anser":ans,"question":qs}
      f.write(json.dumps(datablock))
      f.write("\n")
      id+=1


In [ ]:
n_dec=512 #디코더 모델의 크기
def makebatch(datafile, trainfile, testfile, valfile):
    qs = []
    ans = []
    lines = open(datafile, 'r').readlines()
    train_data, test_data = train_test_split(lines, test_size=0.3, random_state=42)
    test_data, val_data = train_test_split(test_data, test_size=0.2, random_state=42)
    def write_to_file(file, data):
        id=0
        with open(file, "w") as f:
            f.write("[")
            ldata=data[-1]
            for line in data:
                if ord(line[0])==10:
                  continue
                ans = spm.encode_as_ids(line)
                l = len(ans)
                if (l > n_dec - 2):
                    ans = ans[:n_dec - 2]
                ans.insert(0, 2)  # vocab 상 2=bos 3= eos 6=mask
                ans.append(3)
                ran = random.randrange(1, l + 2)
                qs = copy.deepcopy(ans)
                qs[ran] = 6
                datablock = {"id": id, "anser": ans, "question": qs}
                f.write(json.dumps(datablock))
                if line==ldata:
                  f.write("\n")
                else:
                  f.write(",\n")
                id += 1
            f.write("]")
    write_to_file(trainfile, train_data)
    write_to_file(testfile, test_data)
    write_to_file(valfile, val_data)

In [ ]:
makebatch("input.txt","train.json","test.json","val.json")

In [ ]:
lines = open("input.txt", 'r').readlines()
train_data, test_data = train_test_split(lines, test_size=0.3, random_state=42)
test_data, val_data = train_test_split(test_data, test_size=0.2, random_state=42)
if train_data=="\t":
  print(11)
print(ord('\n'))

10
